In [1]:
from xml.etree import ElementTree as ET
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import folium
import pandas as pd
import math

In [11]:
env_path = '.env'

In [12]:
load_dotenv(dotenv_path=env_path)

True

In [139]:
dbname = os.getenv('DB_NAME')
user = os.getenv('DB_USER')
password = os.getenv('DB_PASS')
host = os.getenv('DB_HOST')
port = os.getenv('DB_PORT')

In [140]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [141]:
ns = {'kml': 'http://www.opengis.net/kml/2.2'}

In [142]:
planned_trip_path = '../notebooks/data/1.1 LINHAS RMTC [2023-11-29].kml'

In [143]:
sql_idle_route = """ 
SELECT "data", carro, first_travel, last_travel, geolocalizacao_ida, geolocalizacao_volta
FROM hp.gld_idle_route;
"""

In [144]:
sql_schedule = """ 
SELECT date_execution, linha, carro, re, nome
FROM hp.slv_schedule;
"""

In [145]:
df_idle_route = pd.read_sql_query(sql_idle_route, engine)

In [146]:
def read_and_extract_placemarks(file_path, namespace):
    """ 
    :param: file_path: caminho para o arquivo kml
    :return: lista de dicionários com os nomes e coordenadas dos placemarks

    A função lê um arquivo kml e extrai os placemarks, retornando uma lista de dicionários com os nomes e coordenadas dos placemarks.
    """
    tree = ET.parse(file_path)
    root = tree.getroot()
    placemarks = []
    for placemark in root.findall('.//kml:Placemark', namespace):
        name = placemark.find('kml:name', namespace).text if placemark.find('kml:name', namespace) is not None else "Unnamed"
        coordinates = placemark.find('.//kml:coordinates', namespace).text.strip() if placemark.find('.//kml:coordinates', namespace) is not None else "No coordinates"
        placemarks.append({'name': name, 'coordinates': coordinates})
    return placemarks

In [147]:
placemarks_line_bd = read_and_extract_placemarks(planned_trip_path, ns)

In [148]:
df_base_rota = pd.DataFrame(placemarks_line_bd)

In [149]:
df_base_rota['Código'] = df_base_rota['name'].apply(lambda x: x.split('-', 1)[0])

In [150]:
df_base_rota['identificacao'] = df_base_rota['name'].apply(lambda x: x.split('-', 1)[1])
df_base_rota['identificacao'] = df_base_rota['identificacao'].apply(lambda x: x.split('- ', 1)[0])
df_base_rota['identificacao'] = df_base_rota['identificacao'].apply(lambda x: x.split(' ', 1)[0])

In [151]:
df_base_rota['Código'] = df_base_rota['name'].apply(lambda x: x.split('-', 1)[0])
df_base_rota['Descrição e Sentido'] = df_base_rota['name'].apply(lambda x: x.split(' - ', 1)[1] if ' - ' in x else '')
df_base_rota['Descrição'] = df_base_rota['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[0] if ' - ' in x else x)
df_base_rota['Sentido'] = df_base_rota['Descrição e Sentido'].apply(lambda x: x.rsplit(' - ', 1)[1] if ' - ' in x else '')
df_base_rota.drop('Descrição e Sentido', axis=1, inplace=True)

In [152]:
de_para = {
    '1': 'A', '2': 'B', '3': 'C', '4': 'D', '5': 'E',
    '6': 'F', '7': 'G', '8': 'H', '9': 'I', '21': 'A',
    '22': 'B', '23': 'C', '24': 'D', '25': 'E', '26': 'F',
    '27': 'G', '28': 'H', '29': 'I', '31': 'A', '32': 'B',
    '33': 'C', '34': 'D', '35': 'E', '36': 'F', '37': 'G',
    '38': 'H', '39': 'I'
}

In [153]:
df_base_rota['sublinha'] = df_base_rota['identificacao'].apply(lambda x: de_para[x[:-1]])

In [154]:
df_base_rota['num_linha'] = df_base_rota['Código'].astype(str) + df_base_rota['sublinha']

In [155]:
# Substituir todos os valores "401A" por "401" na coluna "num_linha" do DataFrame df_base_rota
df_base_rota['num_linha'] = df_base_rota['num_linha'].replace('401A', '401')

In [156]:
df_schedule = pd.read_sql_query(sql_schedule, engine)

In [157]:
df_schedule['date_execution'] = pd.to_datetime(df_schedule['date_execution']) 

In [158]:
df_schedule['linha']=df_schedule['linha'].str.strip()

In [159]:
df_schedule[df_schedule['date_execution'] == '2024-04-30'][['linha', 'carro', 're', 'nome']]

,linha,carro,re,nome
102719,333A,20323,000017651,RUI BRASIL LAZARO
103013,326A,20323,000017651,RUI BRASIL LAZARO
103455,706A,20323,000017651,RUI BRASIL LAZARO
118409,117A,1201,000007200,LINDOMAR ANTONIO GARCIA
118410,117A,1201,000007200,LINDOMAR ANTONIO GARCIA
...,...,...,...,...
120529,027A,20998,000018886,EZEQUIEL DA SILVA HUTIM
120530,027B,20998,000017239,RONES SOARES DOS SANTOS
120531,026A,20998,000017239,RONES SOARES DOS SANTOS
120532,026A,20998,000017239,RONES SOARES DOS SANTOS


In [160]:
list_car = df_schedule['carro'].unique().tolist()

In [161]:
sql_made_trip = """ 
SELECT registration, carro, made_trip_coordinates, execution_date
FROM hp.slv_made_trip;

"""

In [162]:
df_made_trip = pd.read_sql_query(sql_made_trip,engine)

In [163]:
df_idle_route['data'] = pd.to_datetime(df_idle_route['data'])

In [164]:
df_made_trip['execution_date'] = pd.to_datetime(df_made_trip['execution_date'])

In [165]:
df_made_trip['carro'] = pd.to_numeric(df_made_trip['carro'])

In [166]:
df_test = pd.merge(df_idle_route, df_made_trip, left_on=['data','carro'], right_on=['execution_date','carro'])

In [167]:
list_line = df_schedule[(df_schedule['carro'] == 20238) & (df_schedule['date_execution'] == '2024-04-30')]['linha'].values.tolist()

In [168]:
list_line

['401', '400A', '015A', '400A', '401', '911B', '400A', '401', '401']

In [169]:
df_filtered = pd.merge(df_idle_route, df_made_trip, left_on=['data','carro'], right_on=['execution_date','carro'])

In [171]:
def convert_coordinates(coords):
    return [(lat, lng) for [lng, lat] in coords]

In [172]:
def parse_coordinates(coord_string):
    coord_string = coord_string.strip()
    coords = coord_string.split(' ')
    parsed_coords = []
    for point in coords:
        if point:
            try:
                components = point.split(',')
                if len(components) >= 2: 
                    lat, lng = components[:2] 
                    parsed_coords.append((float(lat), float(lng)))
            except ValueError as e:
                print(f"Erro ao processar o ponto '{point}': {e}")
                continue  
    
    return parsed_coords

In [173]:
list_car = df_made_trip[df_made_trip['execution_date'] == '2024-04-30']['carro'].unique().tolist()

In [174]:
df_filtered = pd.merge(df_idle_route, df_made_trip, left_on=['data','carro'], right_on=['execution_date','carro'])

In [178]:
TOLERANCIA = 2.5

In [179]:
DISTANCIA_TOLERANCIA = 1 #km 

In [196]:
# Função para calcular a distância entre dois pontos usando a fórmula de Haversine
def haversine(lon1, lat1, lon2, lat2):
    lon1, lat1, lon2, lat2 = map(math.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = math.sin(dlat / 2) ** 2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2) ** 2
    c = 2 * math.asin(math.sqrt(a))
    r = 6371  # Radius of earth in kilometers. Use 3956 for miles
    return c * r



In [ ]:
def parse_coordinates_format_dict(coord_data):
    """
    Função para coordenadas no formato de lista de dicionários: [{'lat': lat, 'lng': lng}, ...]
    """
    try:
        result = [(coord['lat'], coord['lng']) for coord in coord_data]
    except (KeyError, ValueError):
        result = []
    return result



In [ ]:
def parse_coordinates_format_list(coord_data):
    """
    Função para coordenadas no formato de lista de listas: [[lng, lat], ...]
    """
    try:
        result = [(coord[1], coord[0]) for coord in coord_data]
    except (IndexError, ValueError):
        result = []
    return result



In [ ]:
def parse_coordinates_format_str(coord_data):
    """
    Função para coordenadas no formato de string: lng, lat
    """
    try:
        coords = [coord.split(',') for coord in coord_data]
        result = [(float(coord[1]), float(coord[0])) for coord in coords]
    except (IndexError, ValueError):
        result = []
    return result



In [ ]:
def parse_coordinates_format_text(coord_data):
    """
    Função para coordenadas no formato de texto: "lng,lat,alt lng,lat,alt ..."
    """
    try:
        coords_list = coord_data.split(' ')
        parsed_coords = [[float(coord.split(',')[1]), float(coord.split(',')[0])] for coord in coords_list]
        return parsed_coords
    except (IndexError, ValueError):
        return []





In [ ]:
date_execution = '2024-04-30'
def process_car(df_idle_route, df_made_trip, df_schedule, df_base_rota, car):
    df_filtered = pd.merge(df_idle_route, df_made_trip, left_on=['data', 'carro'], right_on=['execution_date', 'carro'])
    df_filtered = df_filtered[(df_filtered['data'] == date_execution) & (df_filtered['carro'] == car)]

    if df_filtered.empty:
        print(f"Nenhum dado para o carro {car} no dia especificado.")
        return

    # Inicializar um mapa
    first_row = df_filtered.iloc[0]
    lat_start = (first_row['geolocalizacao_ida'][0]['lat'] + first_row['geolocalizacao_volta'][0]['lat']) / 2
    lng_start = (first_row['geolocalizacao_ida'][0]['lng'] + first_row['geolocalizacao_volta'][0]['lng']) / 2
    m = folium.Map(location=[lat_start, lng_start], zoom_start=12)

    # Plotar rotas ociosas
    ida = parse_coordinates_format_dict(first_row['geolocalizacao_ida'])
    volta = parse_coordinates_format_dict(first_row['geolocalizacao_volta'])
    folium.PolyLine(ida, color='blue', weight=2.5, opacity=1, tooltip='Rota Ociosa - Ida').add_to(m)
    folium.PolyLine(volta, color='red', weight=2.5, opacity=1, tooltip='Rota Ociosa - Volta').add_to(m)

    # Plotar a rota realizada
    made_trip = parse_coordinates_format_list(first_row['made_trip_coordinates'])
    folium.PolyLine(made_trip, color='green', weight=2.5, opacity=1, tooltip='Rota Realizada').add_to(m)

    # Para cada linha planejada, plotar a rota correspondente
    list_line = df_schedule[(df_schedule['carro'] == car) & (df_schedule['date_execution'] == date_execution)]['linha'].values.tolist()
    for line in list_line:
        print(line)
        df_base_rota_line = df_base_rota[df_base_rota['num_linha'] == line]
        coordinates = df_base_rota_line['coordinates'].iloc[0]
        if isinstance(coordinates, str):
            parsed_coordinates = parse_coordinates_format_text(coordinates)
        else:
            parsed_coordinates = parse_coordinates_format_dict(coordinates)

        # Plotar a rota planejada
        folium.PolyLine(parsed_coordinates, color='purple', weight=2.5, opacity=0.5, dash_array='5, 5',tooltip=f'Linha {line}').add_to(m)

    # Calcular distâncias
    planned_distance = sum([haversine(ida[i][1], ida[i][0], ida[i+1][1], ida[i+1][0]) for i in range(len(ida)-1)])
    planned_distance += sum([haversine(volta[i][1], volta[i][0], volta[i+1][1], volta[i+1][0]) for i in range(len(volta)-1)])
    for route in list_line:
        df_base_rota_line = df_base_rota[df_base_rota['num_linha'] == route]
        coordinates = df_base_rota_line['coordinates'].iloc[0]
        if isinstance(coordinates, str):
            route_coords = parse_coordinates_format_text(coordinates)
        else:
            route_coords = parse_coordinates_format_dict(coordinates)
        planned_distance += sum([haversine(route_coords[i][1], route_coords[i][0], route_coords[i+1][1], route_coords[i+1][0]) for i in range(len(route_coords)-1)])

    real_distance = sum([haversine(made_trip[i][1], made_trip[i][0], made_trip[i+1][1], made_trip[i+1][0]) for i in range(len(made_trip)-1)])

    # Adicionar informações sobre a distância percorrida
    info_html = f"""
    <div style="position: fixed; 
                 bottom: 50px; right: 50px; width: 300px; height: 115px; 
                 border:2px solid grey; z-index:9999; font-size:14px;
                 background-color:white;
                 ">&nbsp; <b>Informações de Distância:</b> <br>
                   &nbsp; Data: {date_execution} <br>
                   &nbsp; Distância Planejada: {planned_distance:.2f} km <br>
                   &nbsp; Distância Realizada: {real_distance:.2f} km <br>
                   &nbsp; Diferença de Distância: {planned_distance - real_distance:.2f} km <br>
 
    </div>
    """
    m.get_root().html.add_child(folium.Element(info_html))
    mapa_filename = f'map_{car}.html'
    m.save(f'./src/{mapa_filename}')
    print(f"Mapa salvo como {mapa_filename}")

In [197]:
for car in list_car:
    print(car)
    process_car(df_idle_route, df_made_trip, df_schedule, df_base_rota, car)

20238
401
400A
015A
400A
401
911B
400A
401
401
Mapa salvo como map_20238.html
20018
401
400A
176A
160A
400A
400A
401
Mapa salvo como map_20018.html
20360
015A
400A
176A
401
911B
400A
Mapa salvo como map_20360.html
20362
915A
401
401
401
401
400A
Mapa salvo como map_20362.html
20021
176A
160A
015A
915B
015A
Mapa salvo como map_20021.html
20224
015A
915A
400A
Mapa salvo como map_20224.html
20048
400A
911A
400A
176A
015A
023A
400A
Mapa salvo como map_20048.html
20019
401
400A
911A
400A
401
911B
911B
160A
403B
Mapa salvo como map_20019.html
20357
015A
015A
176A
160A
176A
400A
160A
Mapa salvo como map_20357.html
